In [1]:
# Models
from sklearn.svm import LinearSVC #SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
# Other Packages
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
# From PDFInterpreter import both PDFResourceManager and PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
# Import this to raise exception whenever text extraction from PDF is not allowed
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.layout import LAParams, LTTextBox, LTTextLine, LTTextBoxHorizontal, LTFigure, LTChar, LTText, LTAnno
from pdfminer.converter import PDFPageAggregator
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.externals import joblib
import random
import re
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
import math
import pickle
import joblib
import json
from gensim.models import KeyedVectors
# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#load existing embeddings
en_model = KeyedVectors.load_word2vec_format('cc.de.300.vec')

In [19]:
#read data with features

data =  pd.read_csv('03_data.csv')

In [20]:
def word_embedding(df):

    #preprocess words
    word_emb_input = pd.DataFrame(df.loc[:,'word'].str.lower())
    word_emb_input = pd.DataFrame(word_emb_input)
    word_emb_input = word_emb_input.astype(str)
    word_emb_input['preprocessed'] = np.nan
    word_emb_input ['preprocessed'] = [re.sub('\d', 'D', x) for x in word_emb_input['word'].tolist()]
    word_emb_input = word_emb_input.drop(['word'],1)

    #create embeddings

    #calculate embeddings for every word
    missing=[0]*300
    def fun(key):
        try:
            return(en_model[key])
        except:
            return(missing)
    word_emb_input['vector'] = word_emb_input['preprocessed'].apply(fun)
    word_emb = pd.DataFrame(word_emb_input['vector'].values.tolist())

    #concat data
    emb_data = pd.concat((df, word_emb), axis=1, sort=False)
    return emb_data

In [21]:
def sliding_window(df):

    # create sliding window

    #Feature Groups
    ort_col = ['doc', 'Page', 'word']
    labels = ['chapter', 'subchapter', 'version', 'directive', 'signal', 'chem', 'company', 'date','usecase']
    paper_feature = ['word.is.lower', 'word.is.upper', 'word.is.mixed.case', 'word.is.digit', 'word.contains.digit', 'word.is.special.char','word.len.1', 'word.len.3', 'word.len.5', 'word.len.7', 'word.len.9', 'word.len.11', 'word.len.13', 'word.is.stop']
    date_specific_feature = ['word.is.print.date.trigger', 'word.is.revision.date.trigger', 'word.is.valid.date.trigger', 'word.is.oldversion.date.trigger']
    new_feature = ['word.is.bold', 'word.is.newline','ycord_average','Xcord_first', 'grid.area_11', 'grid.area_12', 'grid.area_13', 'grid.area_14', 'grid.area_15', 'grid.area_16', 'grid.area_17', 'grid.area_18', 'grid.area_21', 'grid.area_22', 'grid.area_23', 'grid.area_24', 'grid.area_25', 'grid.area_26', 'grid.area_27', 'grid.area_28', 'grid.area_31', 'grid.area_32', 'grid.area_33', 'grid.area_34', 'grid.area_35', 'grid.area_36', 'grid.area_37', 'grid.area_38', 'grid.area_41', 'grid.area_42', 'grid.area_43', 'grid.area_44', 'grid.area_45', 'grid.area_46', 'grid.area_47', 'grid.area_48', 'is.page.1', 'is.page.2', 'is.page.3']

    data_ord = df[ort_col + labels]

    columns = [paper_feature, date_specific_feature, new_feature]

    #create window for all features except word embedding
    window_size_feat = 13
    #copies the previous and following features for every token in a given window
    for col in range (len(columns)):
        sel_col = df[columns[col]]
        data_ord = pd.concat([data_ord, sel_col], axis=1, sort=False)
        for i in range (1, math.ceil(window_size_feat/2)):
            data_pre = sel_col.shift(i).add_prefix ('-' + str(i) + '_')
            data_suc = sel_col.shift(-i).add_prefix ('+' + str(i) + '_')
            data_ord = pd.concat([data_ord, data_pre, data_suc], axis=1, sort=False)

    df.columns = df.columns.astype(str)
    
    emd_col = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
'23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65',
'66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162',
'163', '164', '165', '166', '167', '168', '169', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '220', '221', '222', '223', '224', '225', '226', '227', '228', '229', '230', '231', '232', '233', '234', '235', '236', '237', '238', '239', '240', '241', '242', '243', '244', '245', '246', '247', '248', '249', '250', '251', '252', '253', '254', '255', '256', '257', '258', '259', '260', '261', '262', '263', '264', '265', '266', '267', '268', '269', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '280', '281', '282', '283', '284', '285', '286', '287', '288', '289', '290', '291',
'292', '293', '294', '295', '296', '297', '298','299']

    
    #create window for word embedding
    window_size_emb = 3
    sel_col = df.loc[:,emd_col]
    data_ord = pd.concat([data_ord, sel_col], axis=1, sort=False)
    for i in range (1, math.ceil(window_size_emb/2)):
        data_pre = sel_col.shift(i).add_prefix ('-' + str(i) + '_')
        data_suc = sel_col.shift(-i).add_prefix ('+' + str(i) + '_')
        data_ord = pd.concat([data_ord, data_pre, data_suc], axis=1, sort=False)
        
    return data_ord

In [22]:
data = word_embedding(data)

In [23]:
data = sliding_window(data)

In [24]:
data.to_csv('04_data.csv')